<a href="https://colab.research.google.com/github/agemagician/Prot-Transformers/blob/master/Benchmark/Albert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Benchmark ProtAlbert Model using GPU or CPU <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 3.8MB 13.8MB/s 
     |████████████████████████████████| 1.1MB 44.0MB/s 
     |████████████████████████████████| 890kB 43.1MB/s 


In [2]:
import torch
from transformers import AlbertModel
import time
from datetime import timedelta
import os
import requests
from tqdm.auto import tqdm

<b>2. Set the url location of ProtAlbert and the vocabulary file<b>

In [3]:
modelUrl = 'https://www.dropbox.com/s/gtajtmege43ec7k/pytorch_model.bin?dl=1'
configUrl = 'https://www.dropbox.com/s/me7zsqrnpiz043v/config.json?dl=1'
tokenizerUrl = 'https://www.dropbox.com/s/60mg00r361vth4t/albert_vocab_model.model?dl=1'

<b>3. Download ProtAlbert models and vocabulary files</b>

In [4]:
downloadFolderPath = 'models/ProtAlbert/'

In [5]:
modelFolderPath = downloadFolderPath

modelFilePath = os.path.join(modelFolderPath, 'pytorch_model.bin')

configFilePath = os.path.join(modelFolderPath, 'config.json')

tokenizerFilePath = os.path.join(modelFolderPath, 'spm_model.model')

In [6]:
if not os.path.exists(modelFolderPath):
    os.makedirs(modelFolderPath)

In [7]:
def download_file(url, filename):
  response = requests.get(url, stream=True)
  with tqdm.wrapattr(open(filename, "wb"), "write", miniters=1,
                    total=int(response.headers.get('content-length', 0)),
                    desc=filename) as fout:
      for chunk in response.iter_content(chunk_size=4096):
          fout.write(chunk)

In [8]:
if not os.path.exists(modelFilePath):
    download_file(modelUrl, modelFilePath)

if not os.path.exists(configFilePath):
    download_file(configUrl, configFilePath)

if not os.path.exists(tokenizerFilePath):
    download_file(tokenizerUrl, tokenizerFilePath)

<b>3. Load ProtAlbert Model<b>

In [9]:
model = AlbertModel.from_pretrained(modelFolderPath)

<b>4. Load the model into the GPU if avilabile and switch to inference mode<b>

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [11]:
model = model.to(device)
model = model.eval()

<b>5. Benchmark Configuration<b>

In [12]:
min_batch_size = 8
max_batch_size = 32
inc_batch_size = 8

min_sequence_length = 64
max_sequence_length = 512
inc_sequence_length = 64

iterations = 10

<b>6. Start Benchmarking<b>

In [13]:
device_name = torch.cuda.get_device_name(device.index) if device.type == 'cuda' else 'CPU'

with torch.no_grad():
    print((' Benchmarking using ' + device_name + ' ').center(80, '*'))
    print(' Start '.center(80, '*'))
    for sequence_length in range(min_sequence_length,max_sequence_length+1,inc_sequence_length):
        for batch_size in range(min_batch_size,max_batch_size+1,inc_batch_size):
            start = time.time()
            for i in range(iterations):
                input_ids = torch.randint(1, 20, (batch_size,sequence_length)).to(device)
                results = model(input_ids)[0].cpu().numpy()
            end = time.time()
            ms_per_protein = (end-start)/(iterations*batch_size)
            print('Sequence Length: %4d \t Batch Size: %4d \t Ms per protein %4.2f' %(sequence_length,batch_size,ms_per_protein))
        print(' Done '.center(80, '*'))
    print(' Finished '.center(80, '*'))

******************* Benchmarking using Tesla P100-PCIE-16GB ********************
************************************ Start *************************************
Sequence Length:   64 	 Batch Size:    8 	 Ms per protein 0.04
Sequence Length:   64 	 Batch Size:   16 	 Ms per protein 0.04
Sequence Length:   64 	 Batch Size:   24 	 Ms per protein 0.04
Sequence Length:   64 	 Batch Size:   32 	 Ms per protein 0.04
************************************* Done *************************************
Sequence Length:  128 	 Batch Size:    8 	 Ms per protein 0.09
Sequence Length:  128 	 Batch Size:   16 	 Ms per protein 0.08
Sequence Length:  128 	 Batch Size:   24 	 Ms per protein 0.08
Sequence Length:  128 	 Batch Size:   32 	 Ms per protein 0.08
************************************* Done *************************************
Sequence Length:  192 	 Batch Size:    8 	 Ms per protein 0.13
Sequence Length:  192 	 Batch Size:   16 	 Ms per protein 0.12
Sequence Length:  192 	 Batch Size:   24 	 Ms 